## Uploading Files from Kaggle

In [1]:
! pip install -q kaggle
from google.colab import files



In [2]:
# choose the kaggle.json file that you downloaded
files.upload()
! mkdir ~/.kaggle


Saving kaggle.json to kaggle.json


In [3]:

# make a directory named kaggle and copy the kaggle.json file there
!cp kaggle.json ~/.kaggle/



In [4]:
# change the permissions of the file
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download -d subhankarpanda56/news-bias

100% 110M/110M [00:07<00:00, 16.1MB/s]
100% 110M/110M [00:07<00:00, 14.5MB/s]


In [6]:
! unzip news-bias.zip

Archive:  news-bias.zip
  inflating: new_bias.csv            


## Installing and importing Libraries

In [7]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 691.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.4 MB/s eta 0:00:00


In [8]:
import transformers

from transformers import TFBertForSequenceClassification
from transformers import BertTokenizer


import tensorflow as tf
import pandas as pd
import json
import gc

from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')


import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
df = pd.read_csv("/content/new_bias.csv")
df.head(5)

,topic,source,bias,url,title,date,authors,content,content_original,source_url,bias_text,ID
0,terrorism,New York Times - News,0,http://www.nytimes.com/2016/09/20/nyregion/ahm...,"Bomb Suspect Changed After Trip Abroad, Friend...",2016-09-20,N. R. Kleinfield,"Besides his most recent trip to Quetta , Mr. R...","Besides his most recent trip to Quetta, Mr. Ra...",www.nytimes.com,left,004Gt3gcsotuiYmz
1,supreme_court,Vox,0,https://www.vox.com/policy-and-politics/2018/9...,Why Susan Collins claims she’s being bribed ov...,2018-09-12,"Emily Stewart, Terry Nguyen, Rebecca Jennings,...",Is Maine Republican Sen. Susan Collins being b...,Is Maine Republican Sen. Susan Collins being b...,www.vox.com,left,00eP4XD3VdMmHITE
2,education,Ezra Klein,0,http://www.npr.org/blogs/thetwo-way/2014/05/06...,Poll: Prestigious Colleges Won't Make You Happ...,2014-05-06,Anya Kamenetz,Poll : Prestigious Colleges Wo n't Make You Ha...,Poll: Prestigious Colleges Won't Make You Happ...,www.npr.org,left,00FTGIZEd6B8zQ4U
3,us_house,Breitbart News,2,http://www.breitbart.com/big-government/2017/0...,Paul Ryan Reportedly Says No Chance for Border...,2017-09-12,Ian Mason,"House Speaker Paul Ryan , at a private dinner ...","House Speaker Paul Ryan, at a private dinner e...",www.breitbart.com,right,00HGGqBRf1kzPRlg
4,white_house,Guest Writer - Left,0,https://www.cnn.com/2019/07/11/politics/donald...,OPINION: Trump seeking change of legal fortune...,2019-07-11,Analysis Stephen Collinson,( CNN ) President Donald Trump has reason to h...,(CNN) President Donald Trump has reason to hop...,www.cnn.com,left,00IzI5ynahBVtC9l


In [10]:
import numpy as np
np.random.seed(42)

# Randomly select 12,000 rows from the DataFrame
df1 = df.sample(n=7000)


In [11]:
texts = df1['content'].to_list()
labels = df1['bias'].to_list()

In [12]:
df1.bias.value_counts()

2    2530
0    2421
1    2049
Name: bias, dtype: int64

## Preprocessing

In [18]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size = 0.2, random_state = 0 )


train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.01, random_state = 0 )

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [20]:
train_encodings = tokenizer(train_texts, truncation = True, padding = True  )

val_encodings = tokenizer(val_texts, truncation = True, padding = True )

test_encodings =  tokenizer(test_texts, truncation = True, padding = True )

In [21]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))



In [17]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
! pip install "ray[tune]"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 35.3 MB/s eta 0:00:00


In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 KB 7.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, RobertaConfig, BertTokenizer
from keras_tuner.tuners import RandomSearch

def build_model(hp):
    config = RobertaConfig.from_pretrained('roberta-base', num_labels=3)
    tokenizer = BertTokenizer.from_pretrained('roberta-base')

    learning_rate = hp.Float('learning_rate', min_value=3e-5, max_value=4e-5, sampling='LOG')

    model = TFBertForSequenceClassification(config=config)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

    return model

# Adjust the number of trials and max_epochs as needed
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=1,
    directory='hyperparameter_tuning',
    project_name='roberta_tuning'
)

In [ ]:
tuner.search(train_texts, train_labels, epochs=2, validation_data=(val_texts, val_labels))


Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
3.1383e-05        |3.5352e-05        |learning_rate

Epoch 1/2


Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras_tuner/engine/hypermodel.py", line 144, in fit
    return model.fit(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.wi

RuntimeError: ignored

## Training

In [26]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments

from sklearn.metrics import f1_score




training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=2,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=1e-5,               
    logging_dir='./logs',            
    eval_steps=100,
    fp16 = True,
    evaluation_strategy = "steps",
    learning_rate = 3e-5
)

with training_args.strategy.scope():
    trainer_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


trainer = TFTrainer(
    model=trainer_model,                 
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset,    
          
)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
trainer.train()

In [28]:
predictions = trainer.predict(val_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

(1400, 3) (1400,)


In [29]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [31]:
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.2 MB/s eta 0:00:00


In [36]:

import evaluate

metric = evaluate.load("f1")
metric.compute(predictions=preds, references=predictions.label_ids, average = 'micro')

{'f1': 0.7314285714285713}

## Testing

In [37]:
trainer.evaluate()

{'eval_loss': 0.6178478761152788}

In [38]:
test_result = trainer.predict(test_dataset)
probs = test_result.predictions
preds = probs.argmax(axis=1)

In [39]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, preds)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 73.21%


In [40]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(test_labels, preds)
print("MAE score:", mae)

MAE score: 0.4107142857142857


In [41]:
save_directory = "/saved_models" 

model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

('/saved_models/tokenizer_config.json',
 '/saved_models/special_tokens_map.json',
 '/saved_models/vocab.txt',
 '/saved_models/added_tokens.json')

In [42]:
tokenizer_fine_tuned = BertTokenizer.from_pretrained(save_directory)

model_fine_tuned = TFBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /saved_models were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /saved_models.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [49]:
# News from left

df['bias'][2]

0

In [48]:
predict_input = tokenizer_fine_tuned.encode(
    df['content'][2],
    truncation = True,
    padding = True,
    return_tensors = 'tf'    
)

output = model_fine_tuned(predict_input)[0]

prediction_value = tf.argmax(output, axis = 1).numpy()[0]
                                                      

prediction_value       # 0: Left, 1: Center, 2: Right

0